# Annexe

Dans l'OpenData d'IDFM, en plus des validations sur le réseau ferré, il y a aussi des données de validation sur le réseau de surface, c'est-à-dire les lignes de bus et de tramway. Mais ces données de validations sont différentes de celles du réseau ferré. Pour le réseau ferré nous avions des données sur des stations (La Défense, Châtelet...), alors que pour le réseau de surface nous avons des données sur des lignes (le bus 30, le tramway T2...) car les valideurs sont situés directement à bord des bus ou des tramways. Ces données ne rentrent donc pas dans notre sujet, mais peuvent tout de même montrer des résultats intéressants. On se propose de les étudier dans une annexe.

N. B. : toutes les lignes qui ont un nom commençant par T sont des tramways (sauf le TVM qui est un bus), et par N des noctiliens (bus de nuit).

### Importations

In [ ]:
import pandas as pd
import numpy as np

## I. Validations sur le réseau de surface (RDS)

In [ ]:
df_rds = pd.read_csv("data/validations/validations_rds.csv",sep = ";")
vald_rds = (
    df_rds.groupby(['ID_GROUPOFLIGNE', 'LIBELLE_LIGNE'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)

# Chaque ligne a un identifiant unique : ID_GROUPOFLIGNE

vald_rds['NB_VALD_MOY_JOUR'] = (vald_rds['NB_VALD']/91).round(0) #pour arrondir : round(0)
vald_rds = vald_rds[vald_rds["ID_GROUPOFLIGNE"] != "A00000"] #on enlève les lignes non-définies
vald_rds.head(10)

,ID_GROUPOFLIGNE,LIBELLE_LIGNE,NB_VALD,NB_VALD_MOY_JOUR
981,A01193,T3a,10700314,117586.0
1228,A01757,T3b,10662056,117165.0
980,A01192,T2,8621139,94738.0
1414,A02371,T9 - Paris Porte de Choisy - Orly Gaston Viens,5622812,61789.0
736,A00924,TVM,4912649,53985.0
1268,A02149,T6,4029208,44277.0
763,A00952,62,2776453,30510.0
1269,A02150,T8,2488586,27347.0
1231,A01760,T5,2460480,27038.0
761,A00950,60,1821021,20011.0


Les lignes les plus fréquentées du réseau de surface sont surtout des lignes de tramways, c'est plutôt logique car un tramway peut contenir bien plus de personnes qu'un bus.
La ligne la plus fréquentée du réseau de surface est le T3a, suivie de près par le T3b. Ces deux lignes réunies ont un trajet faisant le tour de Paris. Ce sont les seules lignes de tramways ayant un trajet intégralement intra-muros.

La ligne de bus la plus fréquentée d'Ile de France est le TVM (TVM est pour Trans Val de Marne). Comme son nom l'indique, elle traverse le département du Val de Marne d'Est en Ouest. C'est une des rares lignes d'Ile de France pouvant être qualifiée de BHNS (Bus à Haut Niveau de Service), car elle bénéficie d'un site propre sur la quasi-intégralité de son trajet. C'est-à-dire que les bus roulent sur une voie qui leur exclusivement réservée, et ont la priorité aux intersections. Cela permet de gagner énormément de temps sur le trajet.

Remarque : En plus d'être le bus le plus fréquenté d'Ile de France comme on le constate ici, le TVM est le bus le plus fréquenté d'Europe !

## II. Fréquence des lignes

In [ ]:
# Calcul de l'offre. Dans le GTFS il y a réseau ferré et réseau de surface.
agency = pd.read_csv("data/gtfs/agency.txt") #associe agency_id avec réseau de transport. Go trouver Paris-Saclay
routes = pd.read_csv("data/gtfs/routes.txt") #associe route_id avec une ligne de transport
trips = pd.read_csv("data/gtfs/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id
calendar = pd.read_csv("data/gtfs/calendar.txt") #quel jour il y a tel service, identifié par service_id
calendar_dates = pd.read_csv("data/gtfs/calendar_dates.txt") #services à ajouter/retirer car exceptions
stops = pd.read_csv("data/gtfs/stops.txt") #associe stop_id avec un arrêt physique
stop_times = pd.read_parquet("data/gtfs/stop_times.parquet")

In [ ]:
services_jour = (calendar.groupby(
    ["start_date"],as_index=False)["service_id"].nunique()
) #nombre de services qui commencent tel jour

#On prend le lundi 8 décembre, il a l'air bien au milieu de tout

date = pd.Timestamp("2025-12-08")
dint = int(date.strftime("%Y%m%d")) #conversion au même format que les dates des données GTFS

#Services actifs le 08/12
actifs = calendar[
    (calendar["start_date"] <= dint) &
    (calendar["end_date"]   >= dint) &
    (calendar["monday"] == 1)
]["service_id"]

#Exceptions du 08/12
exceptions = calendar_dates[calendar_dates["date"] == dint]
actifs = pd.Index(actifs).unique()

enlever = pd.Index(exceptions.loc[exceptions["exception_type"] == 2, "service_id"]).unique()
ajouter = pd.Index(exceptions.loc[exceptions["exception_type"] == 1, "service_id"]).unique()
#loc : sert à sélectionner certaines lignes et certaines colonnes, avec potentiellement des booléens

actifs = actifs.difference(enlever)
actifs = actifs.union(ajouter)
d_actifs = {}
for service_id in actifs :
    d_actifs[service_id] = 1

trips_0812 = trips.loc[trips["service_id"].isin(actifs)]
#isin : vérifie pour chaque élément du DataFrame s'il est dans actifs

offre = trips_0812.merge(routes,on="route_id",how="left")
offre = offre[["route_id","route_long_name","trip_id","direction_id"]]

# Heure (ou minute) de départ du trip au premier arrêt
st = stop_times.loc[stop_times["trip_id"].isin(offre["trip_id"])].copy()
st = st.sort_values(["trip_id", "stop_sequence"])
first_dep = st.groupby("trip_id", as_index=False).first()[["trip_id", "departure_time"]]

# Conversion de HH:MM:SS en minutes depuis 00:00 (gère par ex "25:10:00" si nécessaire)
def to_min(t):
    #marche même si t dépasse 24:00:00
    h, m, s = map(int, t.split(":"))
    return h*60 + m + s/60

first_dep["dep_min"] = first_dep["departure_time"].map(to_min)

dep = first_dep.merge(offre[["trip_id","route_id","direction_id"]], on="trip_id", how="left")

# Pointe du matin 7h–10h
peak = dep[(dep["dep_min"] >= 7*60) & (dep["dep_min"] < 10*60)]

freq = (peak.groupby(["route_id","direction_id"], as_index=False)
             .size()
             .rename(columns={"size": "voyages_3h"}))
freq["voyages_h"] = freq["voyages_3h"] / 3.0
freq["headway_min"] = 60.0 / freq["voyages_h"]

freq = freq.merge(
    routes[["route_id","route_type","route_long_name"]],
    on="route_id", how="left"
)
freq = freq.sort_values(by="voyages_h",ascending=False)
#chaque ligne apparait 2 fois dans la liste, une fois pour chaque sens

freq = freq.drop_duplicates(subset="route_long_name",keep="first") 
freq.head(10)

,route_id,direction_id,voyages_3h,voyages_h,headway_min,route_type,route_long_name
2065,IDFM:C01742,1,138,46.000000,1.304348,2,A
2063,IDFM:C01740,1,114,38.000000,1.578947,2,L
2054,IDFM:C01728,1,107,35.666667,1.682243,2,D
1664,IDFM:C01371,1,105,35.000000,1.714286,1,1
1687,IDFM:C01383,0,103,34.333333,1.747573,1,13
2066,IDFM:C01743,1,102,34.000000,1.764706,2,B
2059,IDFM:C01737,1,102,34.000000,1.764706,2,H
2062,IDFM:C01739,1,101,33.666667,1.782178,2,J
2053,IDFM:C01727,1,100,33.333333,1.800000,2,C
1689,IDFM:C01384,0,100,33.333333,1.800000,1,14


Les lignes les plus fréquentes en heure de pointe du matin en semaine ne sont que des lignes ferrées. On peut l'expliquer que beaucoup de lignes ferrées d'Ile de France transportent énormément de voyageurs, ce qui nécessite une grande fréquence. Fréquence rendue possible par le fait d'avoir des voies dédiées à une unique ligne. La ligne la plus fréquente est le RER A, avec en moyenne un train toutes les 1.3 minutes sur le tronçon central.

Restreignons maintenant au réseau de surface :

In [ ]:
freq_rds = freq[(freq["route_type"] == 0) | (freq["route_type"] == 3)]
freq_rds.head(15)

,route_id,direction_id,voyages_3h,voyages_h,headway_min,route_type,route_long_name
1139,IDFM:C01071,0,62,20.666667,2.903226,3,TVM
1272,IDFM:C01142,1,54,18.000000,3.333333,3,113
1701,IDFM:C01390,0,52,17.333333,3.461538,0,T2
2160,IDFM:C01843,0,50,16.666667,3.600000,0,T4
1703,IDFM:C01391,0,48,16.000000,3.750000,0,T3a
1238,IDFM:C01123,1,47,15.666667,3.829787,3,92
1993,IDFM:C01679,0,47,15.666667,3.829787,0,T3b
2110,IDFM:C01794,1,47,15.666667,3.829787,0,T6
1510,IDFM:C01266,1,46,15.333333,3.913043,3,291
1252,IDFM:C01132,1,45,15.000000,4.000000,3,103


Nous remarquons la présence de pas mal de lignes de tramway. Dans la même logique que les lignes ferrées, ces lignes transportent énormément de personnes donc doivent être très fréquentes, et bénéficient d'une voie dédiée. Cependant, la fréquence maximale d'une ligne de bus est plus élevée qu'une ligne de tramway, car le fait que le tramway soit contraint à rester sur des voies rend l'exploitation plus difficile. Il est par exemple impossible d'avoir deux tramways qui se suivent l'un collé à l'autre, alors que pour des bus cela ne pose pas de problème. 
C'est pour cette raison que la ligne la plus fréquente du réseau de surface est le TVM, qui combine à la fois le fait d'être un bus, et le fait d'avoir un site propre.

Autre remarque que nous pouvons faire sur ce classement : nous remarquons la présence de 5 lignes de bus exploitées avec des bus standards (113, 92, 291, 103, 129) et de 4 avec des bus articulés (TVM, Caméléon T1, 4606, 258). 
En effet, comme les bus standards peuvent transporter moins de personnes que les bus articulés, ils nécessitent une fréquence plus élevée sur les lignes très fréquentées.
(A notre connaissance il n'existe pas de base de données qui répertorie le matériel roulant de chaque ligne de bus, ces informations viennent de la page Wikipédia des lignes de bus RATP.)

Enfin, en tant qu'étudiants du Plateau de Saclay, nous constatons que la ligne 4606, que (presque) tout étudiant de l'ENSAE a déjà prise au moins une fois, est la 5ème ligne de bus la plus fréquente d'Ile de France en heure de pointe avec un bus tous les 4 minutes en moyenne. Cette ligne relie les campus universitaires et les logements du plateau à la gare de Massy-Palaiseau, point de passage quasiment indispensable pour se rendre à Paris depuis le plateau.
On peut rajouter que le tracé du 4606 va être entièrement repris par la ligne 18 du Grand Paris Express, qui devrait être mise en service fin 2026, avec moins de points d'arrêts. Le 4606 risque donc de voir son offre baisser et de ne plus être dans ce classement d'ici 1 an.

En effet, rappelons que dans ce classement nous regardons la fréquence de ces lignes en heure de pointe. Ainsi, la ligne 4606 n'est pas impactée par sa fréquence d'un bus tous les 15 minutes en heure creuse.

Etudions maintenant le réseau de bus de Paris-Saclay, car c'est le secteur de l'ENSAE, donc d'une certaine manière cela nous concerne directement !

In [ ]:
freq_routes = freq.merge(routes,on="route_id",how="left")
freq_saclay = freq_routes[freq_routes["agency_id"] == "IDFM:1060"] #restriction à Paris-Saclay
freq_saclay.head(10)

Nous remarquons deux lignes : le 4606 qu'on a déjà vu avant, ainsi que le 4609. Cette ligne complète la desserte du plateau de Saclay, en reliant le campus d'HEC et le secteur de l'université Paris-Saclay à la gare du Guichet.
Certaines lignes ont des fréquences très basses car ce sont des lignes scolaires. Elles sont dédiées à une demande très ciblée.

## III. Lien avec les POI et limites

Nous nous demandons maintenant : est-ce que les lignes de bus et de tramway qui desservent beaucoup de POI sont les plus fréquentées ?

Pour chaque ligne, nous allons prendre un trip représentatif et relier chaque stop du trip par un segment, puis tracer un buffer de 200 mètres autour de ces segments.

Dans certains GTFS il y a un fichier shapes.txt qui donne les formes exactes des lignes et qui nous aurait évité d'avoir à approximer avec des segments les tracés des lignes, mais il n'y en a pas dans le GTFS d'IDFM. Plus une ligne a de distance entre chacun de ses arrêts, moins l'approximation est bonne.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from pathlib import Path

def build_bus_tram_buffers_fast(
    gtfs_dir,
    distance=200,
    agency_filter=None  # ex: ['RATP']
):

    gtfs_dir = Path(gtfs_dir)

    # --- 1. Lecture ---
    routes = pd.read_csv(gtfs_dir / "routes.txt")
    trips = pd.read_csv(gtfs_dir / "trips.txt")
    stops = pd.read_csv(gtfs_dir / "stops.txt")
    stop_times = pd.read_parquet(gtfs_dir / "stop_times.parquet")

    # --- 2. Filtrer bus + tram ---
    routes_bt = routes[routes["route_type"].isin([0, 3])].copy()

    if agency_filter is not None and "agency_id" in routes_bt.columns:
        routes_bt = routes_bt[routes_bt["agency_id"].isin(agency_filter)].copy()

    route_ids_bt = set(routes_bt["route_id"])
    trips_bt = trips[trips["route_id"].isin(route_ids_bt)].copy()

    # Gérer absence de direction_id
    if "direction_id" not in trips_bt.columns:
        trips_bt["direction_id"] = 0

    # --- 3. Trip représentatif par (route_id, direction_id) ---
    reps = (
        trips_bt
        .sort_values(["route_id", "direction_id", "trip_id"])
        .groupby(["route_id", "direction_id"])
        .head(1)
    )

    rep_trip_ids = set(reps["trip_id"])
    stop_times_rep = stop_times[stop_times["trip_id"].isin(rep_trip_ids)].copy()

    # --- 4. Ajouter coordonnées ---
    stops_coords = stops[["stop_id", "stop_lon", "stop_lat"]]
    st_rep = stop_times_rep.merge(stops_coords, on="stop_id", how="left")

    # --- 4bis. Ajouter route_id & direction_id AVANT tri ---
    st_rep = st_rep.merge(
        reps[["trip_id", "route_id", "direction_id"]],
        on="trip_id",
        how="left"
    )

    # 4ter. Trier maintenant correctement 
    st_rep = st_rep.sort_values(
        ["route_id", "direction_id", "trip_id", "stop_sequence"]
    )

    # 5. Construire LineString par (route_id, direction_id) 
    lines = []
    for (route_id, direction_id), df_grp in st_rep.groupby(["route_id", "direction_id"]):
        df_grp = df_grp.dropna(subset=["stop_lon", "stop_lat"])
        if len(df_grp) < 2:
            continue

        coords = list(zip(df_grp["stop_lon"], df_grp["stop_lat"]))
        line = LineString(coords)
        lines.append({
            "route_id": route_id,
            "direction_id": direction_id,
            "geometry": line
        })

    if not lines:
        return gpd.GeoDataFrame(columns=["route_id", "direction_id", "geometry"], crs="EPSG:2154")

    gdf_lines = gpd.GeoDataFrame(lines, crs="EPSG:4326")

    # 6. Projeter en Lambert 93 & buffer
    gdf_lines_2154 = gdf_lines.to_crs("EPSG:2154")
    gdf_lines_2154["geometry"] = gdf_lines_2154.geometry.buffer(distance)

    # 7. Calculer l'aire couverte par chaque ligne
    gdf_lines_2154["area_m2"] = gdf_lines_2154.geometry.area

    return gdf_lines_2154


In [ ]:
import numpy as np

def poi_par_ligne(gtfs_dir, poi_path, buffers, distance=200):

    # 2) lire les POI
    poi = gpd.read_parquet(poi_path)

    # Mettre les POI en EPSG:2154
    if poi.crs is None or poi.crs.to_string() != "EPSG:2154":
        poi = poi.set_crs("EPSG:4326", allow_override=True).to_crs("EPSG:2154")

    # 3) jointure spatiale POI ↔ buffers de lignes
    hits = gpd.sjoin(
        poi[["geometry"]],
        buffers[["route_id", "geometry"]],
        how="inner",
        predicate="within"  # ou "intersects"
    )

    # donner un identifiant unique aux POI si besoin
    hits["poi_id"] = np.arange(len(hits))

    # 4) compter les POI (uniques) par route_id
    poi_par_route = (
        hits.groupby("route_id")["poi_id"]
        .nunique()
        .reset_index()
        .rename(columns={"poi_id": "nb_poi_{}m".format(distance)})
    )

    # Jointure pour récupérer l'aire
    df_lignes = poi_par_route.merge(
        buffers[["route_id", "direction_id", "area_m2"]],
        on="route_id",
        how="left"
    )

    # Indicateur : POI par km² de bande de desserte
    df_lignes["poi_par_m2"] = df_lignes["nb_poi_200m"] / df_lignes["area_m2"]

    return df_lignes

In [ ]:
# Construction des DataFrame

gtfs_dir = "data/gtfs"
buffers = build_bus_tram_buffers_fast(gtfs_dir, distance=200)
liste_poi_surface = []

for path in ["poi_administration2.parquet","poi_bureaux2.parquet","poi_commerce_majeur.parquet"
,"poi_commerce_proximite.parquet","poi_culture.parquet","poi_education2.parquet","poi_sports.parquet",
"poi_sante2.parquet","poi_logement.parquet"] :
    poi_surface = poi_par_ligne(gtfs_dir, "data/poi/" + path, buffers, distance=200)
    poi_surface = poi_surface.sort_values(by="poi_par_m2",ascending=False)
    poi_surface = poi_surface.merge(routes,on="route_id",how="inner")
    poi_surface = poi_surface[["route_id","route_long_name","nb_poi_200m","poi_par_m2"]]
    poi_surface = poi_surface.drop_duplicates(subset=["route_id"], keep="first") #certaines lignes ont plusieurs trips représentatifs
    liste_poi_surface.append(poi_surface)


Une autre limite de cette méthode est que pour gagner du temps, on ne prend qu'un ou deux trips représentatifs par ligne pour tracer les segments, donc pour les lignes qui ne passent pas aux mêmes endroits dans les deux sens cela réduit sûrement beaucoup leur aire, le petit rayon de 200 mètres ne compense pas.

In [ ]:
labels = ["admin2","bureaux2","commaj","comprox","culture","educ2","sports","sante2","logement"]

# Extraire tous les route_id présents au moins dans un des fichiers
all_route_ids = pd.concat([df[["route_id","route_long_name"]] for df in liste_poi_surface]).drop_duplicates()

# Unification en un seul DataFrame
df_merged = all_route_ids.copy()

for df, label in zip(liste_poi_surface, labels):
    tmp = df[["route_id", "nb_poi_200m", "poi_par_m2"]].copy()
    tmp = tmp.rename(columns={
        "nb_poi_200m": f"nb_poi_200m_{label}",
        "poi_par_m2": f"poi_par_m2_{label}",
    })
    df_merged = df_merged.merge(tmp, on="route_id", how="left")
df_merged = df_merged.drop_duplicates(subset=["route_id"], keep="first")
df_merged = df_merged.fillna(0)

df_merged.head(5)

In [ ]:
# Lien avec les validations

ref_lignes = pd.read_csv("data/ref/ref_lignes.csv",sep=";")
ref_lignes["route_id"] = "IDFM:" + ref_lignes["ID_Line"]
ref_lignes = ref_lignes[["route_id","ID_GroupOfLines"]]

df_merged = df_merged.merge(ref_lignes,on="route_id",how="left")

poi_vald_surface = df_merged.merge(vald_rds,left_on="ID_GroupOfLines",right_on="ID_GROUPOFLIGNE",how="left")
poi_vald_surface.sort_values(by="NB_VALD_MOY_JOUR").head(10)

Avant de faire une régression des validations sur les poi, on enlève les noctiliens qui faussent tout avec leur amplitude horaire réduite, alors que leurs trajets sont très longs et traversent des zones denses

In [ ]:
filtre = poi_vald_surface[~poi_vald_surface["route_long_name"].str.contains("N", na=False)] # ~ inverse la condition

# remplacer inf / -inf par NaN
filtre = filtre.replace([np.inf, -np.inf], np.nan)

# supprimer toutes les lignes contenant un NaN
filtre = filtre.dropna()

X = filtre[["nb_poi_200m_admin2","nb_poi_200m_bureaux2","nb_poi_200m_commaj","nb_poi_200m_comprox","nb_poi_200m_culture",
"nb_poi_200m_educ2","nb_poi_200m_sports","nb_poi_200m_sante2","nb_poi_200m_logement"]]
y = filtre["NB_VALD_MOY_JOUR"]

model_poi = sm.OLS(y, X).fit()
print(model_poi.summary())

On obtient un R² de XX. Donc les POI expliquent moins bien la fréquentation des lignes de bus que des lignes ferrées, avec les modèles que l'on a fait.

Les lignes de bus sont très nombreuses et très variées, et d'autres variables que l'on ne peut pas prendre en compte avec nos données semblent pouvoir expliquer leur fréquentation.

Par exemple, en reprenant le classement des lignes du réseau de surface les plus fréquentées, dans le top10 apparaissent deux lignes de bus parisiennes : le 62 et le 60. Ces deux lignes desservent des zones géographiquement différentes (le Sud pour le 62 et le Nord-Est pour le 60) mais quand on regarde leur trajet, ces deux lignes ont un point commun : elles desservent des zones mal desservies par le métro et ont un trajet en rocade. Alors que le réseau de métro de Paris est principalement radial (un problème que va tenter de résoudre la ligne 15 du Grand Paris Express), le 62 et le 60 permettent des trajets qui sont contraignants à faire en métro. Au vu de la très forte densité du réseau de métro parisien, ces cas sont rares.
Ces deux lignes ne desservent pas de grand pôle, elles répondent surtout à une demande résidentielle qui n'a pas de meilleure alternative.

Ainsi, un critère que l'on pourrait prendre en compte pour expliquer la fréquentation des lignes de bus serait le manque d'alternatives ferrées. Cependant, cela semble difficile à mesurer.